In [1]:
import pandas as pd
import ms3 
from  ms3.utils import *
import os 
from ms3 import Score
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### Convert to mscx if not done yet

In [37]:
path_midi = '/home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/'
target = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/'

dir_list = os.listdir(path_midi)

for el in dir_list:
    convert(path_midi+el, target+el[:-4]+'.mscx', MS='musescore3')

INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/Crazy little thing called love-Queen.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/Crazy little thing called love-Queen.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/Come_As_You_Are-Nirvana.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/Come_As_You_Are-Nirvana.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/I Shot the Sheriff-Eric Clapton.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/I Shot the Sheriff-Eric Clapton.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/Hey Jude- Beatles.mid to /home/nulpe/Desktop/

### Mscx to rythm vectors including all instruments

In [11]:
def rythm_vectors(in_dir, out_dir):
    list_sheet_music = os.listdir(in_dir)
    df_rythm_vectors =[]

    for idx, el in enumerate(list_sheet_music):
        print(el[-4:])
        if el[-4:] == 'mscx':

            
            #Get notes with onsets
            s = Score(dir_sheet_music+el)
            df = s.mscx.notes

            # Define instruments 
            instrument_dict = {}
            for key in s.mscx.metadata['parts']:
                for staff in s.mscx.metadata['parts'][key].keys():
                    instrument_dict[staff] = key


            #staff to instruments 
            df['instrument'] = [instrument_dict[el] if el in instrument_dict else 'na' for el in df.staff]


            # define quarter beat 
            df['quarter_beats'] = (df.mc_onset*16).astype('int32')


            #make rythm matrix & data frame
            df_histogram = df.groupby(['instrument','quarter_beats'])['mn'].agg(['count'])
            df_histogram = df_histogram.groupby(level=0).apply(lambda x: x.reset_index(level = 0).drop(['instrument'],axis=1).reindex(list(range(0,16)),fill_value=0).T)
            df_histogram = df_histogram.reset_index()

            df_histogram.insert(loc=0, column='song_artist', value=el[:-5])

            #concat to big rythm vector df
            if len(df_rythm_vectors) == 0: df_rythm_vectors = df_histogram

            df_rythm_vectors = pd.concat([df_rythm_vectors,df_histogram], axis=0)

    df_rythm_vectors.to_csv(out_dir, index = False)

    
dir_sheet_music = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/mscx_tresillos/'
out_dir = '/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillio.csv'
rythm_vectors(dir_sheet_music, out_dir)

mscz
mscx
mscx
mscx
mscz
mscx
mscz
mscx


In [40]:
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv')
df_rythm_vectors

,song_artist,instrument,level_1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"Eastside-benny blanco, Halsey, Khalid",Athmosphere,count,32,0,0,0,0,0,0,0,30,0,0,0,30,0,0,0
1,"Eastside-benny blanco, Halsey, Khalid",Bright Piano,count,64,0,0,0,0,0,0,0,32,0,0,0,60,0,0,0
2,"Eastside-benny blanco, Halsey, Khalid",Fingered Bass,count,16,0,0,16,16,0,0,0,15,0,0,15,15,0,7,0
3,"Eastside-benny blanco, Halsey, Khalid",Jazz Guitar,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
4,"Eastside-benny blanco, Halsey, Khalid",Nylon Gtr.,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul","Syn.Strings 1, Cheap Thrills",count,36,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0
84,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Synth Voice,count,12,0,0,0,6,0,6,0,12,0,0,0,6,0,0,0
85,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Tenor Sax,count,18,2,11,4,11,0,9,0,17,5,4,2,11,3,7,4
86,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Voice Oohs,count,1,0,0,0,0,0,10,0,11,0,10,0,10,0,0,0


def collapse_normalize_vectors():
    rythm_vector_collaped = df_rythm_vectors.groupby(['song_artist']).agg(['sum'])
    rythm_vector_collaped.columns = rythm_vector_collaped.columns.droplevel(1)
    rythm_vector_collaped = rythm_vector_collaped.drop(['instrument', 'level_1'],axis=1)
    rythm_vector_collaped["sum"] = rythm_vector_collaped.sum(axis=1)
    rythm_vector_collaped = rythm_vector_collaped.loc[:,"0":"15"].div(rythm_vector_collaped["sum"], axis=0)
    return rythm_vector_collaped### collapse instruments, one vector per song

In [55]:
def collapse_normalize_vectors(df_rythm_vectors):
    rythm_vector_collaped = df_rythm_vectors.groupby(['song_artist']).agg(['sum'])
    rythm_vector_collaped.columns = rythm_vector_collaped.columns.droplevel(1)
    rythm_vector_collaped = rythm_vector_collaped.drop(['instrument', 'level_1'],axis=1)
    rythm_vector_collaped["sum"] = rythm_vector_collaped.sum(axis=1)
    rythm_vector_collaped = rythm_vector_collaped.loc[:,"0":"15"].div(rythm_vector_collaped["sum"], axis=0)
    return rythm_vector_collaped

In [57]:
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv')
rythm_vector_collaped = collapse_normalize_vectors(df_rythm_vectors)
rythm_vector_collaped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
song_artist,,,,,,,,,,,,,,,,
"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",0.211900,0.007307,0.015136,0.102296,0.062109,0.000000,0.105428,0.000000,0.199374,0.008873,0.016701,0.103862,0.067328,0.001566,0.096033,0.002088
"Eastside-benny blanco, Halsey, Khalid",0.221595,0.006351,0.019054,0.064220,0.042343,0.014820,0.052929,0.010586,0.172900,0.007763,0.019760,0.062103,0.187721,0.022583,0.062809,0.032463
"Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber",0.151248,0.004607,0.122073,0.013820,0.119386,0.004607,0.099808,0.002687,0.110173,0.003071,0.101727,0.007678,0.122073,0.008829,0.116699,0.011516
New rules-Dua Lipa,0.176933,0.020533,0.050240,0.092180,0.061162,0.014417,0.050677,0.010922,0.097859,0.015291,0.076453,0.121014,0.082569,0.015291,0.102665,0.011796
"Rockabye-Clean Bandit, Sean Paul, Anne-Marie",0.183642,0.003086,0.052469,0.075617,0.043981,0.008488,0.075617,0.037037,0.154321,0.033951,0.038580,0.090278,0.053241,0.023148,0.105710,0.020833
Shape of you-Ed Sheran,0.288392,0.004326,0.041096,0.003605,0.059841,0.025955,0.144196,0.022350,0.098053,0.007931,0.038212,0.007931,0.225667,0.016583,0.014420,0.001442
Treat you better-Shawn Mendes,0.150000,0.023656,0.036559,0.086559,0.064516,0.024731,0.085484,0.023118,0.138710,0.023118,0.029570,0.087634,0.067204,0.032796,0.095699,0.030645


In [48]:
df_synt_tresillo = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillio.csv')
df_synt_tresillo = df_synt_tresillo[df_synt_tresillo['song_artist']!='Raggetone'].drop(['instrument','level_1'], axis=1)
df_synt_tresillo.index = df_synt_tresillo.song_artist
df_synt_tresillo.drop(['song_artist'], axis=1)
df_synt_tresillo["sum"] = df_synt_tresillo.sum(axis=1)
df_synt_tresillo = df_synt_tresillo.loc[:,"0":"15"].div(df_synt_tresillo["sum"], axis=0)
tresillo_vector = df_synt_tresillo.sum()/len(df_synt_tresillo)
tresillo_vector = np.asarray(tresillo_vector).reshape(-1, 1).T

In [63]:
def calc_cosine_sim(rythm_vectors, tresillo_vector):
    rythm_vectors['cosine_sim_tresillo'] = cosine_similarity(rythm_vectors.loc[:,"0":"15"],tresillo_vector)
    return rythm_vectors

rythm_vector_collaped = calc_cosine_sim(rythm_vector_collaped, tresillo_vector)
rythm_vector_collaped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,cosine_sim_tresillo
song_artist,,,,,,,,,,,,,,,,,
"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",0.211900,0.007307,0.015136,0.102296,0.062109,0.000000,0.105428,0.000000,0.199374,0.008873,0.016701,0.103862,0.067328,0.001566,0.096033,0.002088,0.913635
"Eastside-benny blanco, Halsey, Khalid",0.221595,0.006351,0.019054,0.064220,0.042343,0.014820,0.052929,0.010586,0.172900,0.007763,0.019760,0.062103,0.187721,0.022583,0.062809,0.032463,0.803275
"Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber",0.151248,0.004607,0.122073,0.013820,0.119386,0.004607,0.099808,0.002687,0.110173,0.003071,0.101727,0.007678,0.122073,0.008829,0.116699,0.011516,0.774479
New rules-Dua Lipa,0.176933,0.020533,0.050240,0.092180,0.061162,0.014417,0.050677,0.010922,0.097859,0.015291,0.076453,0.121014,0.082569,0.015291,0.102665,0.011796,0.926529
"Rockabye-Clean Bandit, Sean Paul, Anne-Marie",0.183642,0.003086,0.052469,0.075617,0.043981,0.008488,0.075617,0.037037,0.154321,0.033951,0.038580,0.090278,0.053241,0.023148,0.105710,0.020833,0.915665
Shape of you-Ed Sheran,0.288392,0.004326,0.041096,0.003605,0.059841,0.025955,0.144196,0.022350,0.098053,0.007931,0.038212,0.007931,0.225667,0.016583,0.014420,0.001442,0.700478
Treat you better-Shawn Mendes,0.150000,0.023656,0.036559,0.086559,0.064516,0.024731,0.085484,0.023118,0.138710,0.023118,0.029570,0.087634,0.067204,0.032796,0.095699,0.030645,0.940530


In [66]:
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_no_tresillos_4_4.csv')
non_tresillio_vectors = collapse_normalize_vectors(df_rythm_vectors)
non_tresillio_vectors = calc_cosine_sim(non_tresillio_vectors, tresillo_vector)
non_tresillio_vectors['cosine_sim_tresillo']

song_artist
A hard days night-Beatles               0.794264
Cocaine-Eric Clapton                    0.789114
Come together-Beatles                   0.778343
Come_As_You_Are-Nirvana                 0.771313
Crazy little thing called love-Queen    0.718345
Hey Jude- Beatles                       0.697940
I Shot the Sheriff-Eric Clapton         0.909999
Walk of Life- Dire Straits              0.708941
Name: cosine_sim_tresillo, dtype: float64

In [75]:
df_synt_tresillo

for el in df_synt_tresillo.iterrows():
    vector = np.asarray(el).reshape(-1, 1).T
    print(vector)
    #print(calc_cosine_sim(non_tresillio_vectors, vector)['cosine_sim_tresillo'])

[['Cinquillo'
  0     0.1
1     0.0
2     0.1
3     0.1
4     0.0
5     0.1
6     0.1
7     0.0
8     0.1
9     0.0
10    0.1
11    0.1
12    0.0
13    0.1
14    0.1
15    0.0
Name: Cinquillo, dtype: float64]]
[['Three side_two side'
  0     0.16
1     0.00
2     0.04
3     0.16
4     0.00
5     0.04
6     0.16
7     0.00
8     0.04
9     0.00
10    0.16
11    0.04
12    0.12
13    0.04
14    0.04
15    0.00
Name: Three side_two side, dtype: float64]]
[['Vanilla_Tresillo'
  0     0.166667
1     0.000000
2     0.000000
3     0.166667
4     0.000000
5     0.000000
6     0.166667
7     0.000000
8     0.166667
9     0.000000
10    0.000000
11    0.166667
12    0.000000
13    0.000000
14    0.166667
15    0.000000
Name: Vanilla_Tresillo, dtype: float64]]
[['Tresillo over two'
  0     0.125
1     0.000
2     0.000
3     0.125
4     0.125
5     0.000
6     0.125
7     0.000
8     0.125
9     0.000
10    0.000
11    0.125
12    0.125
13    0.000
14    0.125
15    0.000
Name: Tresillo over two,